In [ ]:
using DataFrames
using JLD2
using PlotlyJS
using Interact

In [ ]:
include("plot_utils.jl")

In [ ]:
@load "Data/roi_data_ind.jld2" rois

Recalculate effecive time constant

In [ ]:
@. rois[!, :τw] = (rois.τL * abs(rois.L) + rois.τR * abs(rois.R))/(abs(rois.L)+abs(rois.R));

Spatial coordinates and <1px jitter to improve display

In [ ]:
dx = dy = 0.8f0
dz = -1.0f0
jitter = randn(Float32, (size(rois.x,1),3));

In [ ]:
using Interact

In [ ]:
function change_display(to_display)
    if to_display == "τ effective"
        colors = continuous_color_by_parameter(rois.τw, param_max=8, hue_min=65,
 hue_max=0, luminance_min=75, luminance_max=20, chroma_min=70, chroma_max=70)
    elseif to_display == "P"
        colors = continuous_color_by_parameter(log.(rois.P),param_min=-0.6, param_max=0.6, hue_min=120,
  hue_max=320, luminance_min=50, luminance_max=30, chroma_min=70, chroma_max=2)
    elseif to_display == "τ difference"
        colors = continuous_color_by_parameter(abs.(rois.τL .- rois.τR), param_max=8, hue_min=280,
              hue_max=200, luminance_min=70, luminance_max=30, chroma_min=40, chroma_max=60)
    elseif to_display == "ei_θ"
        colors = continuous_colors_from_rois(rois, l_start=48,l_end=48)
    end
    restyle!(p, marker_color=[colors])
    restyle!(paramplot, marker_color=[colors])
end

function change_markersize(ms)
    restyle!(p, marker_size=ms)
end

dd = dropdown(["τ effective","ei_θ", "τ difference", "P"])
Interact.@on change_display(&dd)

markersize = slider(0.1:0.1:3, value=1.5, label="dot size")
Interact.@on change_markersize(&markersize)

In [ ]:
ROIs = scatter3d(x=rois.x .* dx.+jitter[:,1],
                 y=rois.y .* dy.+jitter[:,2],
                 z=rois.z .* dz.+jitter[:,3],
                 mode="markers", marker=attr(size=2))
p = plot(ROIs);

paramscatt = scattergl(x=rois.τw,
                     y=rois.ei_θ,
                 mode="markers", marker=attr(size=2))
paramplot = plot(paramscatt);

The dropdown selects the parameter to color the ROIs. Make sure to run the lower cell as well to get a display of the parameter space

In [ ]:
vbox(hbox(dd, markersize), p)

In [ ]:
paramplot

The ei_θ parameter is defined like this:
![](theta_explainer.svg)